In [2]:
import numpy as np
import cv2
import pickle

## 1. Loading the Data containing face features of celebrities

In [3]:
data = pickle.load(open('data_face_features.pickle', mode='rb'))

In [4]:
data

{'data': [array([[ 0.03055908,  0.14285783, -0.14262319, -0.02234345, -0.02406834,
           0.22193357, -0.0087749 , -0.04322924,  0.01046508,  0.00720519,
           0.17144123,  0.07745533,  0.11674768, -0.12279233, -0.1141531 ,
          -0.05186126, -0.07458735,  0.05420615, -0.15217824,  0.12724258,
           0.05768387, -0.02233355, -0.00215821,  0.11978265, -0.07567442,
          -0.02604712,  0.00392897, -0.08837277,  0.03066526,  0.06980011,
           0.03212453,  0.01989333, -0.035154  ,  0.11789224, -0.07252357,
           0.08713815, -0.01925867,  0.03287129, -0.05841968,  0.07487863,
           0.03002623, -0.1404155 , -0.06452198,  0.11661776, -0.1299316 ,
          -0.15533228,  0.12917802,  0.07212506, -0.12651072,  0.12956809,
           0.03237963,  0.03496858, -0.15939018,  0.03534684,  0.03698716,
           0.06935564, -0.1063802 ,  0.10658987, -0.02281427, -0.07044771,
          -0.16542105, -0.01680502,  0.28556296, -0.17745477,  0.01145272,
          -0.1012

In [5]:
X = np.array(data['data']) #independent variable
y = np.array(data['label']) # dependent variable

In [6]:
X.shape, y.shape

((1660, 1, 128), (1660,))

In [7]:
X = X.reshape(-1, 128)
X.shape

(1660, 128)

In [8]:
# Splitting the data into train and test
from sklearn.model_selection import train_test_split

In [9]:
x_train, x_test, y_train, y_test = train_test_split(X,y, train_size=0.8, random_state=42)

In [10]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1328, 128), (1328,), (332, 128), (332,))

## 2. Training the model

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score

### 1. Logistic regression

In [11]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train, y_train) # Training logistic regression

LogisticRegression()

In [23]:
def get_report(model, x_train,y_train,x_test,y_test):
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # Accuracy score
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)

    # f1 score
    f1_score_train = f1_score(y_train, y_pred_train,average='macro')
    f1_score_test = f1_score(y_test, y_pred_test,average='macro')


    print("Accuracy Train = {:.2f},\nAccuracy test = {:.2f},\nf1_score_train = {:.2f},\nf1_score_test = {:.2f}".format(acc_train,acc_test,f1_score_train,f1_score_test))

In [24]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.73,
Accuracy test = 0.71,
f1_score_train = 0.73,
f1_score_test = 0.71


### 2. Support vector machine

In [25]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [26]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.84,
Accuracy test = 0.72,
f1_score_train = 0.83,
f1_score_test = 0.72


### 3. Random forest 

In [67]:
model_rf = RandomForestClassifier()
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=200)

In [68]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00,
Accuracy test = 0.68,
f1_score_train = 1.00,
f1_score_test = 0.68


## 3. Stacking using Voting Classifier 

In [72]:
model_voting = VotingClassifier(estimators=[
    ('logistic', LogisticRegression()),
    ('SVM', SVC(probability=True)),
    ('RF', RandomForestClassifier())
], voting='soft', weights=[2,3,1])

In [73]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('SVM', SVC(probability=True)),
                             ('RF', RandomForestClassifier(n_estimators=200))],
                 voting='soft', weights=[2, 3, 1])

In [74]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.89,
Accuracy test = 0.71,
f1_score_train = 0.89,
f1_score_test = 0.71


## 4. Tune the parameters

In [75]:
from sklearn.model_selection import GridSearchCV

In [76]:
model_grid = GridSearchCV(estimator=model_voting,
                         param_grid={
                             'SVM__C':[3,5,7,10],
                             'SVM__gamma':[0.1,0.3,0.5],
                             'RF__n_estimators':[5,10,20,30],
                             'RF__max_depth':[3,5,7,10],
                             'voting':['soft','hard']
                         }, scoring='accuracy', cv=5, n_jobs=1, verbose=2)

In [77]:
model_grid.fit(x_train,y_train)

Fitting 5 folds for each of 384 candidates, totalling 1920 fits
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.5s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.3s


[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=5, SVM__C=7, SVM__gam

[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.1, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.1, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   0.7s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=5

[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.5, voting=hard; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.5, voting=hard; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.1s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.2s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.1s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.2s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM_

[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   0.8s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   0.9s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.0s
[CV] END RF__max_depth=3, RF__n_estimators=20, SVM__C=7

...

[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.0s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.0s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   0.9s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.1, voting=soft; total time=   1.0s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.1, voting=soft; total time=   0.9s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.1, voting=soft; total time=   1.0s
[CV] END RF__max_depth=10, RF__n_estimators=10, SVM__C=10, SVM__gamma=0.1, voting=soft; total time=   0.9s
[CV] END RF__max_depth=10, RF__n_estimator

[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=soft; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.5, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=20, SVM__C=5, SVM__gamma=0.5, voting=soft; total time=   1.0s
[CV] END RF__max_depth=10, RF__n_estimators=20

[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=soft; total time=   1.3s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=3, SVM__gamma=0.1, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30

[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.3, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=soft; total time=   1.2s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.1s
[CV] END RF__max_depth=10, RF__n_estimators=30, SVM__C=7, SVM__gamma=0.5, voting=hard; total time=   1.3s
[CV] END RF__max_depth=10, RF__n_estimators=30

GridSearchCV(cv=5,
             estimator=VotingClassifier(estimators=[('logistic',
                                                     LogisticRegression()),
                                                    ('SVM',
                                                     SVC(probability=True)),
                                                    ('RF',
                                                     RandomForestClassifier(n_estimators=200))],
                                        voting='soft', weights=[2, 3, 1]),
             n_jobs=1,
             param_grid={'RF__max_depth': [3, 5, 7, 10],
                         'RF__n_estimators': [5, 10, 20, 30],
                         'SVM__C': [3, 5, 7, 10], 'SVM__gamma': [0.1, 0.3, 0.5],
                         'voting': ['soft', 'hard']},
             scoring='accuracy', verbose=2)

In [79]:
model_best_estimator = model_grid.best_estimator_

In [81]:
model_best_score = model_grid.best_score_

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('SVM', SVC(C=10, gamma=0.5, probability=True)),
                             ('RF',
                              RandomForestClassifier(max_depth=10,
                                                     n_estimators=30))],
                 voting='soft', weights=[2, 3, 1])

## 5. Saving the model

In [83]:
pickle.dump(model_best_estimator, open('models/machine_learning_face_person_identity.pkl', mode="wb"))